In [ ]:
# install require module
! pip install nltk
! pip install streamlit
! pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# import require module
import random
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

In [3]:
# download require word tokenizer
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# get  the data set as much as to train the module
intents = [
    {
        "tag": "greeting",
        "patterns": [
            "Hi", "Hello", "Hey", "How are you", "What's up", "Good morning", 
            "Good afternoon", "Good evening", "Hi there", "Hello there", 
            "Hey there", "How's it going", "How have you been", "Nice to see you", 
            "What’s new", "How’s everything", "How’s your day"
        ],
        "responses": [
            "Hi there!", "Hello!", "Hey!", "I'm fine, thank you!", 
            "Nothing much. How about you?", "Good to see you!", 
            "How can I help you today?", "Nice to meet you!"
        ]
    },
    {
        "tag": "goodbye",
        "patterns": [
            "Bye", "See you later", "Goodbye", "Take care", "Have a nice day", 
            "See you soon", "Farewell", "Catch you later", "Bye for now", 
            "Until next time", "Goodnight", "See ya", "Take it easy"
        ],
        "responses": [
            "Goodbye!", "See you later!", "Take care!", "Have a great day!", 
            "See you soon!", "Farewell!", "Catch you later!", "Bye for now!", 
            "Until next time!", "Goodnight!", "See ya!", "Take it easy!"
        ]
    },
    {
        "tag": "thanks",
        "patterns": [
            "Thank you", "Thanks", "Thanks a lot", "I appreciate it", 
            "Thank you so much", "Thanks a ton", "Many thanks", 
            "Thanks for your help", "I’m grateful", "Thank you kindly", 
            "Cheers", "Thanks a bunch", "Much obliged"
        ],
        "responses": [
            "You're welcome!", "No problem!", "Glad I could help!", 
            "My pleasure!", "Anytime!", "Happy to help!", "You’re very welcome!", 
            "Don’t mention it!", "It was nothing!", "Cheers!"
        ]
    },
    {
        "tag": "about",
        "patterns": [
            "What can you do", "Who are you", "What are you", 
            "What is your purpose", "Tell me about yourself", 
            "What do you do", "What’s your function", 
            "Why were you created", "What’s your job", 
            "What’s your role", "What’s your mission"
        ],
        "responses": [
            "I am a chatbot!", "My purpose is to assist you.", 
            "I can answer questions and provide assistance.", 
            "I’m here to help you with your queries.", 
            "I’m a virtual assistant designed to make your life easier.", 
            "I’m here to chat and help you with anything you need."
        ]
    },
    {
        "tag": "help",
        "patterns": [
            "Help", "I need help", "Can you help me", "What should I do", 
            "I need assistance", "Can you assist me", "What can I do", 
            "How can you help me", "I’m stuck", "I need guidance", 
            "Help me out", "I’m confused", "What do you suggest"
        ],
        "responses": [
            "Sure, what do you need help with?", "I'm here to help. What's the problem?", 
            "How can I assist you?", "Tell me what you need help with.", 
            "I’m here to assist you. What’s the issue?", 
            "Let me know how I can help.", "I’m all ears. What’s troubling you?"
        ]
    },
    {
        "tag": "fallback",
        "patterns": [],
        "responses": [
            "I’m sorry, I didn’t understand that. Can you rephrase?", 
            "I’m not sure I follow. Could you clarify?", 
            "I didn’t get that. Can you say it differently?", 
            "I’m still learning. Could you explain that again?"
        ]
    },
    {
        "tag": "jokes",
        "patterns": [
            "Tell me a joke", "Can you tell me a joke", "Say something funny", 
            "Do you know any jokes", "Make me laugh", "Tell me something humorous", 
            "Give me a funny story", "I need a laugh", "Cheer me up with a joke"
        ],
        "responses": [
            "Why don’t scientists trust atoms? Because they make up everything!", 
            "Why did the scarecrow win an award? Because he was outstanding in his field!", 
            "What do you call fake spaghetti? An impasta!", 
            "Why don’t skeletons fight each other? They don’t have the guts!", 
            "What do you call cheese that isn’t yours? Nacho cheese!", 
            "Why did the bicycle fall over? Because it was two-tired!"
        ]
    },
    {
        "tag": "time",
        "patterns": [
            "What time is it", "Tell me the time", "What’s the current time", 
            "Do you know the time", "Can you tell me the time", "What’s the time now", 
            "Time please", "What’s the clock say", "How late is it"
        ],
        "responses": [
            "I’m sorry, I don’t have access to real-time data.", 
            "You can check the time on your device or watch.", 
            "I’m not connected to a clock, but you can check your phone!", 
            "I don’t have the ability to tell time, but your phone does!"
        ]
    },
    {
        "tag": "date",
        "patterns": [
            "What’s today’s date", "What is the date today", "Tell me the date", 
            "What’s the current date", "Can you tell me the date", "What day is it today", 
            "What’s the calendar say", "What’s today’s day and date"
        ],
        "responses": [
            "I’m sorry, I don’t have access to real-time data.", 
            "You can check the date on your device or calendar.", 
            "I’m not connected to a calendar, but you can check your phone!", 
            "I don’t have the ability to tell the date, but your phone does!"
        ]
    },
    {
        "tag": "compliments",
        "patterns": [
            "You’re amazing", "You’re awesome", "You’re the best", 
            "I like you", "You’re so smart", "You’re very helpful", 
            "You’re great", "You’re fantastic", "You’re wonderful", 
            "You’re cool", "I love you"
        ],
        "responses": [
            "Thank you! You’re amazing too!", 
            "Aww, you’re making me blush!", 
            "Thanks! You’re pretty awesome yourself!", 
            "You’re too kind!", 
            "I appreciate that! You’re the best!", 
            "Thank you! I’m here to help!"
        ]
    },
    {
        "tag": "food",
        "patterns": [
            "What’s your favorite food", "Do you eat food", "What do you like to eat", 
            "Can you cook", "What’s the best food", "What should I eat", 
            "What’s for dinner", "What’s a good recipe", "Do you like pizza", 
            "What’s your favorite cuisine"
        ],
        "responses": [
            "I’m a chatbot, so I don’t eat food, but I hear pizza is amazing!", 
            "I don’t eat, but I can help you find a great recipe!", 
            "You should try pizza or pasta—they’re always a good choice!", 
            "I’m not a foodie, but I can recommend some popular dishes!", 
            "I don’t have a favorite food, but I’ve heard sushi is delicious!"
        ]
    },
    {
        "tag": "music",
        "patterns": [
            "What’s your favorite song", "Do you like music", "What music do you like", 
            "Can you recommend a song", "What’s a good playlist", "What’s your favorite band", 
            "Do you listen to music", "What’s the best genre", "What’s your favorite artist"
        ],
        "responses": [
            "I don’t listen to music, but I’ve heard that pop and rock are great genres!", 
            "I can’t listen to music, but I can recommend some popular songs!", 
            "I don’t have a favorite song, but I’ve heard that ‘Bohemian Rhapsody’ is a classic!", 
            "I’m not into music, but you should check out Spotify for great playlists!"
        ]
    },
    {
        "tag": "sports",
        "patterns": [
            "Do you like sports", "What’s your favorite sport", "Who’s your favorite team", 
            "Do you watch football", "What’s the best sport", "Do you play sports", 
            "What’s your favorite athlete", "Do you like basketball", "What’s the most popular sport"
        ],
        "responses": [
            "I don’t play sports, but I’ve heard soccer is the most popular sport in the world!", 
            "I don’t have a favorite team, but I know many people love football!", 
            "I’m not into sports, but I can help you find the latest scores!", 
            "I don’t watch sports, but I’ve heard basketball is exciting!"
        ]
    },
    {
        "tag": "news",
        "patterns": [
            "What’s the news", "Tell me the latest news", "What’s happening in the world", 
            "What’s the top story", "Any breaking news", "What’s going on", 
            "What’s the latest update", "What’s trending", "What’s in the headlines"
        ],
        "responses": [
            "I’m sorry, I don’t have access to real-time news updates.", 
            "You can check the news on a trusted news website or app.", 
            "I don’t have the latest news, but you can check Google News!", 
            "I’m not connected to a news source, but I recommend checking BBC or CNN."
        ]
    },
    {
        "tag": "hobbies",
        "patterns": [
            "What are your hobbies", "What do you like to do", "Do you have any hobbies", 
            "What do you do for fun", "What’s your favorite pastime", 
            "What do you enjoy doing", "How do you spend your time", 
            "What’s your favorite activity"
        ],
        "responses": [
            "I’m a chatbot, so I don’t have hobbies, but I love helping you!", 
            "I don’t have hobbies, but I enjoy chatting with you!", 
            "I’m always here to help, so that’s my favorite activity!", 
            "I don’t have hobbies, but I’m here to assist you with yours!"
        ]
    },
    {
        "tag": "feelings",
        "patterns": [
            "How are you feeling", "Are you happy", "Are you sad", 
            "Do you have feelings", "What’s your mood", "Are you okay", 
            "How’s your day going", "Are you feeling good", "Do you feel emotions"
        ],
        "responses": [
            "I’m a chatbot, so I don’t have feelings, but I’m here to help!", 
            "I don’t feel emotions, but I’m always ready to assist you!", 
            "I’m just a program, but I’m here to make your day better!", 
            "I don’t have feelings, but I’m always happy to help!"
        ]
    }

    
]

In [6]:
# Prepare data for training
all_patterns = []
all_labels = []
responses = {}

for intent in intents:
    for pattern in intent['patterns']:
        all_patterns.append(pattern)
        all_labels.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

# Function to clean and tokenize sentences
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in sentence_words]
    return " ".join(sentence_words)  # Return as a single string

# Vectorize patterns using TF-IDF
vectorizer = TfidfVectorizer(tokenizer=clean_up_sentence)
X = vectorizer.fit_transform(all_patterns)

# Train a Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X, all_labels)

# Function to get a response based on user input
def get_response(user_input):
    input_vector = vectorizer.transform([user_input])
    predicted_proba = classifier.predict_proba(input_vector)[0]
    confidence = np.max(predicted_proba)
    predicted_tag = classifier.predict(input_vector)[0]
    return random.choice(responses[predicted_tag])

# Chat loop
print("Bot: Hello! How can I assist you today? (Type 'quit' to exit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Bot: Goodbye!")
        break
    response = get_response(user_input)
    print(f"Bot: {response}")

/home/codespace/.local/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bot: Hello! How can I assist you today? (Type 'quit' to exit)
Bot: Nothing much. How about you?
Bot: Why did the scarecrow win an award? Because he was outstanding in his field!
Bot: Why did the scarecrow win an award? Because he was outstanding in his field!
Bot: I’m here to chat and help you with anything you need.
Bot: I don’t have the latest news, but you can check Google News!
Bot: Hey!
Bot: Hey!
Bot: Hello!
Bot: I don’t watch sports, but I’ve heard basketball is exciting!
Bot: I don’t have a favorite team, but I know many people love football!
Bot: Good to see you!
Bot: I'm fine, thank you!
Bot: Hey!
Bot: Nothing much. How about you?
Bot: Hi there!
Bot: Nothing much. How about you?
Bot: Hello!
Bot: Hi there!
Bot: I'm fine, thank you!
Bot: Hello!
Bot: Hi there!
Bot: Nothing much. How about you?
Bot: Hello!
Bot: Hey!
Bot: I don’t eat, but I can help you find a great recipe!
Bot: Goodbye!
